Libaries Install:

`pip install matplotlib==3.3.2`

`pip install scipy==1.4.1`


In [30]:
# import the libraries
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [31]:
print("Running Path, use this to set your file path")
print(os.getcwd())

Running Path, use this to set your file path
c:\Users\kunal\Documents\AI-21\ai-21\PROJECTS\voice_recognition


In [32]:
local_file_path = './audioFiles/3.wav'

# Read the wav file (mono)
samplingFrequency, signalData = wavfile.read(local_file_path)
print("Sampling Frequency: {}".format(samplingFrequency))
print("Signal Data: {}".format(signalData))
print("lmao kunal trash")

Sampling Frequency: 8000
Signal Data: [   0    0    0 ...  -21 -222  -35]
lmao kunal trash


Now that we see that the signal data is in a ___ format. We can see how we can manipulate it to measure the variance.

Since we can't print it, lets see how long it is

In [33]:
print(len(signalData))
# What does this mean?

128448


In [34]:
print(len(signalData)/samplingFrequency)

16.056


In [35]:
positionValue = 29700
print("Signal Data for {} : {}".format(positionValue, signalData[positionValue]))

Signal Data for 29700 : 7


# Creating a threshold

In [36]:
# Threshold for all values that are between 50 and -50
sumNon = 0
ct = 0
for i in signalData:
    if not -50 < i < 50 : 
        sumNon+=abs(i)
        ct+=1
print(sumNon/ct)

1139.2267973240366


In [37]:
# Average of all values
sum(abs(signalData))/len(signalData)

667.0591834828101

In [38]:
totalSumNonZero = 0
ct = 0
for num in signalData:
    if num != 0:
        totalSumNonZero+=num
        ct+=1
averageNonZero = totalSumNonZero / ct
print("Average Non Zero: {}".format(averageNonZero))

Average Non Zero: 0.036234905400621485


In [39]:
totalSumNonZero = 0
ct = 0
for num in signalData:
    if num > 10:
        totalSumNonZero+=num
        ct+=1
averageNonZero = totalSumNonZero / ct
print("Average Non Zero: {}".format(averageNonZero))

Average Non Zero: 777.5411818099301


# Apply the threshold

In [40]:
signalDataNormalized = []
for i in signalData:
    if abs(i) > 2000:
        signalDataNormalized.append(i)
    else:
        signalDataNormalized.append(0) 

In [41]:
positionsSpeak = []
runningStartPositionCt = 0
startedSpeaking = False
for pos in range(len(signalDataNormalized)):
    if abs(signalDataNormalized[pos]) > 1999 and startedSpeaking == False:
        runningStartPositionCt = pos
        startedSpeaking = True
    if (startedSpeaking == True and 
        [a for a in signalDataNormalized[pos:pos+100]] == 0):
        positionsSpeak.append([runningStartPositionCt, pos])
        startedSpeaking = False
    runningStartPositionCt+=1

In [42]:
positionsSpeak

[]

In [43]:
idPoints = []
ct = 0
for i in signalDataNormalized:
    if i != 0:
        idPoints.append(ct)
    ct+=1

In [13]:
print("All the points in the following list display all the x values\
       where the y value is greater than the threhsold: " + str(idPoints[0:5]))

All the points in the following list display all the x values       where the y value is greater than the threhsold: [4094, 4095, 4096, 4097, 4098]


In [44]:
# Add code for finding the time in between 

runningCurrentStart = 0
startedPeriod = False
for i in idPoints:
    if i > runningCurrentStart and not startedPeriod:
        runningCurrentStart = i
        startedPeriod = True
    if startedPeriod:
        if runningCurrentStart + 1 > i:
            print("Start: {} End: {}".format(runningCurrentStart, i))
            startedPeriod = False
            runningCurrentStart = 0
        print("Running Current Start: {}".format(runningCurrentStart))

Start: 4094 End: 4094
Running Current Start: 0
Start: 4095 End: 4095
Running Current Start: 0
Start: 4096 End: 4096
Running Current Start: 0
Start: 4097 End: 4097
Running Current Start: 0
Start: 4098 End: 4098
Running Current Start: 0
Start: 4099 End: 4099
Running Current Start: 0
Start: 4100 End: 4100
Running Current Start: 0
Start: 4101 End: 4101
Running Current Start: 0
Start: 4102 End: 4102
Running Current Start: 0
Start: 4103 End: 4103
Running Current Start: 0
Start: 4107 End: 4107
Running Current Start: 0
Start: 4108 End: 4108
Running Current Start: 0
Start: 4109 End: 4109
Running Current Start: 0
Start: 4110 End: 4110
Running Current Start: 0
Start: 4111 End: 4111
Running Current Start: 0
Start: 4112 End: 4112
Running Current Start: 0
Start: 4113 End: 4113
Running Current Start: 0
Start: 4114 End: 4114
Running Current Start: 0
Start: 4115 End: 4115
Running Current Start: 0
Start: 4116 End: 4116
Running Current Start: 0
Start: 4117 End: 4117
Running Current Start: 0
Start: 4118 E